In [2]:
!pip install pandas openpyxl

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/6c/1c/689c9d99bc4e5d366a5fd871f0bcdee98a6581e240f96b78d2d08f103774/pandas-2.0.3-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Obtaining dependency information for numpy>=1.20.3 from https://files.pythonhosted.org/packages/0f/a8/5057b97c395a710999b5697ffedd648caee82c24a29595952d26bd750155/numpy-1.25.2-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 29.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 30.8 MB/s eta 0:00:0000:0100:01


In [5]:
import pandas as pd

# Read the Excel file
file_path = 'dimension_play_log.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

df.columns = [col.lower().replace(' ', '_') for col in df.columns]


# Check the duplicate columns and rename them
cols = df.columns.tolist()
seen = {}
for idx, col in enumerate(cols):
    if col in seen:
        seen[col] += 1
        cols[idx] = f"{col}_{seen[col]}"
    else:
        seen[col] = 0

df.columns = cols

# get rid of the period
df.columns = [col.lower().replace('.', '_') for col in df.columns]
# Display the DataFrame to verify
df.head()


,timestamp,tasks,player_name,a,b,c,d,e,f,g,...,g.3,h.3,i.3,j.3,k.3,l.3,m.3,n.3,score.3,bonus.3
0,2023-08-24 13:29:14.587,"""QUANTITY-2-B"", ""NOTTOP-O"", ""TOUCH-K-K"", ""TOUC...",Andrew,Green,Black,Green,Orange,Orange,Orange,Green,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-24 13:43:33.112,"""NOTOUCH-G-G"", ""TOUCH-G-W"", ""TOUCH-G-K"", ""NOTO...",Andrew,Green,Black,Blue,Blue,White,Green,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-08-24 13:49:28.987,"""QUANTITY-1-K"", ""QUANTITY-1-B"", ""NOTTOP-K"", ""T...",Andrew,White,White,Black,White,Orange,Blue,Orange,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-08-24 13:54:45.847,"""QUANTITY-1-W"", ""QUANTITY-2-O"", ""NOTOUCH-G-K"",...",Andrew,Blue,Green,Green,Green,Blue,Black,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-08-24 14:02:42.970,"""NOTTOP-G"", ""NOTTOP-O"", ""NOTOUCH-K-K"", ""TOUCH-...",Andrew,Blue,White,Black,White,Black,White,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(df.columns.tolist())

['timestamp', 'tasks', 'player_name', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'score', 'bonus', 'player_name_1', 'a_1', 'b_1', 'c_1', 'd_1', 'e_1', 'f_1', 'g_1', 'h_1', 'i_1', 'j_1', 'k_1', 'l_1', 'm_1', 'n_1', 'score_1', 'bonus_1', 'player_name_2', 'a_2', 'b_2', 'c_2', 'd_2', 'e_2', 'f_2', 'g_2', 'h_2', 'i_2', 'j_2', 'k_2', 'l_2', 'm_2', 'n_2', 'score_2', 'bonus_2', 'player_name_3', 'a_3', 'b_3', 'c_3', 'd_3', 'e_3', 'f_3', 'g_3', 'h_3', 'i_3', 'j_3', 'k_3', 'l_3', 'm_3', 'n_3', 'score_3', 'bonus_3']


In [9]:
# Melt the data into a player, turn, task list per row
# List to store mini dataframes
dfs = []

# Split and transform for each player group
for i in range(4):  # Here 4 is the number of player groups you have (0 to 3)
    if i == 0:
        prefix = ''
    else:
        prefix = f'_{i}'
    
    # Filter out rows where player name is empty
    player_df = df[df[f'player_name{prefix}'] != '']

    # Select columns for this player
    cols = ['timestamp', 'tasks', f'player_name{prefix}', f'score{prefix}', f'bonus{prefix}'] + [f'{char}{prefix}' for char in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n']]
    
    subset = player_df[cols]
    
    # Rename columns
    new_cols = ['timestamp', 'tasks', 'name', 'score', 'bonus'] + list('abcdefghijklmn')
    subset.columns = new_cols
    
    dfs.append(subset)

# Combine all mini dataframes
result = pd.concat(dfs, axis=0).reset_index(drop=True)

# print(result)
# print(result.iloc[40]) #get one row


                 timestamp                                              tasks  \
0  2023-08-24 13:29:14.587  "QUANTITY-2-B", "NOTTOP-O", "TOUCH-K-K", "TOUC...   
1  2023-08-24 13:43:33.112  "NOTOUCH-G-G", "TOUCH-G-W", "TOUCH-G-K", "NOTO...   
2  2023-08-24 13:49:28.987  "QUANTITY-1-K", "QUANTITY-1-B", "NOTTOP-K", "T...   
3  2023-08-24 13:54:45.847  "QUANTITY-1-W", "QUANTITY-2-O", "NOTOUCH-G-K",...   
4  2023-08-24 14:02:42.970  "NOTTOP-G", "NOTTOP-O", "NOTOUCH-K-K", "TOUCH-...   
..                     ...                                                ...   
75 2023-08-24 15:50:53.870  "QUANTITY-1-W", "QUANTITY-2-O", "TOUCH-B-B", "...   
76 2023-08-24 15:54:19.995  "QUANTITY-1-G", "QUANTITY-2-G", "NOTTOP-B", "N...   
77 2023-08-24 15:57:04.084  "QUANTITY-1-O", "TOUCH-O-W", "TOUCH-O-K", "NOT...   
78 2023-08-24 15:59:42.983  "NOTTOP-G", "NOTTOP-W", "NOTOUCH-O-O", "TOUCH-...   
79 2023-08-24 16:01:32.730  "QUANTITY-2-W", "NOTTOP-K", "NOTTOP-W", "NOTTO...   

      name  score bonus    

In [25]:
#drop all Nan rows by Name
result = result[result['name'].notna()]


In [26]:
print(result)

                 timestamp                                              tasks  \
0  2023-08-24 13:29:14.587  "QUANTITY-2-B", "NOTTOP-O", "TOUCH-K-K", "TOUC...   
1  2023-08-24 13:43:33.112  "NOTOUCH-G-G", "TOUCH-G-W", "TOUCH-G-K", "NOTO...   
2  2023-08-24 13:49:28.987  "QUANTITY-1-K", "QUANTITY-1-B", "NOTTOP-K", "T...   
3  2023-08-24 13:54:45.847  "QUANTITY-1-W", "QUANTITY-2-O", "NOTOUCH-G-K",...   
4  2023-08-24 14:02:42.970  "NOTTOP-G", "NOTTOP-O", "NOTOUCH-K-K", "TOUCH-...   
5  2023-08-24 14:11:01.800  "QUANTITY-1-G", "QUANTITY-1-O", "NOTOUCH-G-B",...   
6  2023-08-24 14:16:29.344  "QUANTITY-2-G", "QUANTITY-2-B", "TOP-W", "NOTT...   
7  2023-08-24 14:23:09.076  "TOP-K", "TOP-B", "TOUCH-G-G", "TOUCH-B-B", "N...   
8  2023-08-24 15:28:14.012  "NOTTOP-B", "TOUCH-O-O", "TOUCH-W-W", "TOUCH-O...   
9  2023-08-24 15:32:10.361  "QUANTITY-2-W", "NOTOUCH-W-W", "TOUCH-G-O", "T...   
10 2023-08-24 15:36:35.922  "TOP-G", "TOUCH-O-O", "NOTOUCH-G-G", "TOUCH-K-...   
11 2023-08-24 15:39:53.987  